<a href="https://colab.research.google.com/github/RogerHeederer/ForAllPytorch/blob/main/CNN_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [36]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [37]:
#Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [38]:
#Mnist dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform = transforms.ToTensor(),
                         download=True)

In [40]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [76]:
#CNN 모델 만들기
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5

        #Image (1, 1, 28, 28) # batch 1, 채널 1, height 28, width 28
        # Layer1에 들어가는 이미지 = (1, 1, 28, 28)
        # Conv 레이어 통과 후 -> (?, 32, 28, 28) ?:배치
        # pool 레이어 통과 후 -> (?, 32, 14, 14) ?:배치
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1), # 1채널을 받아 32채널로 만든다.
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2에 들어가는 이미지 = (?, 32, 14, 14)
        # Conv 레이어 통과 후 -> (?, 64, 14, 14)
        # pool 레이어 통과 후 -> (?, 64, 7, 7)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # Layer3에 들어가는 이미지 = (?, 64, 7, 7)
        # Conv 레이어 통과 후 -> (?, 128, 7, 7)
        # pool 레이어 통과 후 -> (?, 128, 4, 4)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        )
        # Layer4 는 FC1 레이어. 
        # 위에서 만들어진 (128 * 4 * 4)을 input 넣고 Flatten해서 625 형태로 outputs 해준다.
        self.fc1 = torch.nn.Linear(128 * 4 * 4, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight) # 레이어 웨이트 초기화
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob)
        )

        self.fc2 = torch.nn.Linear(625, 10, bias=True) #625 인풋 받아서 10개로 뿌려줌
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
      out = self.layer1(x)
      out = self.layer2(out)
      out = self.layer3(out)
      out = out.view(out.size(0), -1) #out.size(0)= 배치 사이즈 / Flatten them for FC
      out = self.layer4(out)
      out = self.fc2(out)
      return out

In [77]:
model = CNN().to(device)
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2048, out_features=625, bias=True)
  (layer4): Sequential(
    (0): Linear(in_features=2048, out_features=625, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)

임의 값 넣어서 우선 모델 테스트

In [79]:
value = torch.Tensor(1,1,28, 28).to(device)
print(model(value).shape)
print(model(value))

torch.Size([1, 10])
tensor([[-1.7800e+33,  2.3573e+33,  7.1827e+33, -1.1041e+33,  5.7754e+33,
         -3.6955e+33,  2.4657e+33,  6.3851e+32, -7.6373e+33,  8.5179e+32]],
       grad_fn=<AddmmBackward>)


In [81]:
# loss function and optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [82]:
# 모델 훈련
total_batch = len(data_loader)
model.train()
print('학습 시작')
for epoch in range(training_epochs): #0~14 epoch
    avg_cost = 0

    for X, Y in data_loader:
      #image is already size of 28x28, no reshape
      #label is not one-hot encoded
      X = X.to(device)
      Y = Y.to(device)

      optimizer.zero_grad()
      hypothesis = model(X)
      cost = criterion(hypothesis, Y)
      cost.backward()
      optimizer.step()
      #배치 100개씩 들어가서 cost 구하고, avg_cost에 담고, 이 과정 쭉 반복이후에 마지막에 total_batch로 나눠줌
      avg_cost += cost / total_batch 

    print('Epoch:{:>4}, cost = {:>.9}'.format(epoch+1, avg_cost))
  
print('Learning Finished')

학습 시작
Epoch:   1, cost = 0.197677523
Epoch:   2, cost = 0.0527819954
Epoch:   3, cost = 0.0378717445
Epoch:   4, cost = 0.0292379614
Epoch:   5, cost = 0.0252907258
Epoch:   6, cost = 0.0201752447
Epoch:   7, cost = 0.0188019797
Epoch:   8, cost = 0.0144441584
Epoch:   9, cost = 0.0113046477
Epoch:  10, cost = 0.0121627096
Epoch:  11, cost = 0.0107967518
Epoch:  12, cost = 0.00967978965
Epoch:  13, cost = 0.00828595366
Epoch:  14, cost = 0.0074187736
Epoch:  15, cost = 0.00869606435
Learning Finished


In [88]:
with torch.no_grad():
  model.eval() # set the model to evaluation mode (dropout=False)

  X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.targets.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

Accuracy: 0.986299991607666
